In [ ]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00


In [ ]:
pip install sentence-transformers

In [ ]:
import torch
from torch_geometric.datasets import MovieLens100K

In [ ]:
dataset = MovieLens100K(root='/tmp/MovieLens')
print(dataset)

Extracting /tmp/MovieLens/ml-100k.zip
Processing...


MovieLens100K()


Done!


In [ ]:
data = dataset[0]

# Explore the data
print(data)

HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)


In [ ]:
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import HeteroData


if 'edge_label_index' in data['user', 'rates', 'movie'] and 'edge_label' in data['user', 'rates', 'movie']:
    del data['user', 'rates', 'movie'].edge_label_index
    del data['user', 'rates', 'movie'].edge_label

# Verify the dataset after cleaning
print("Dataset after removing edge_label and edge_label_index:")
print(data)


splitter = RandomLinkSplit(
    num_val=0.15,  # 15% validation edges (reduced for more training data)
    num_test=0.15,  # 15% test edges
    is_undirected=True,  # Ensure this matches the graph type
    add_negative_train_samples=True,  # Enable negative sampling
    neg_sampling_ratio=1.0,  # Ratio of negative samples to positive samples
    edge_types=('user', 'rates', 'movie'),  # Specify the edge type to split
    rev_edge_types=('movie', 'rated_by', 'user'),  # Reverse edge type
    disjoint_train_ratio=0.1  # Ensure disjoint splits between train/val/test
)

# Perform the split
train_data, val_data, test_data = splitter(data)

# Print details about the split
print("\n--- Split Summary ---")
print(f"Training Data Edge Index Shape: {train_data['user', 'rates', 'movie'].edge_index.shape}")
print(f"Validation Data Edge Index Shape: {val_data['user', 'rates', 'movie'].edge_index.shape}")
print(f"Test Data Edge Index Shape: {test_data['user', 'rates', 'movie'].edge_index.shape}")

# Add additional checks if needed
print(f"\nTraining Data Edge Index: {train_data['user', 'rates', 'movie'].edge_index}")
print(f"Validation Data Edge Index: {val_data['user', 'rates', 'movie'].edge_index}")
print(f"Test Data Edge Index: {test_data['user', 'rates', 'movie'].edge_index}")


Dataset after removing edge_label and edge_label_index:
HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)

--- Split Summary ---
Training Data Edge Index Shape: torch.Size([2, 50400])
Validation Data Edge Index Shape: torch.Size([2, 56000])
Test Data Edge Index Shape: torch.Size([2, 68000])

Training Data Edge Index: tensor([[845, 710, 602,  ...,   6, 770, 297],
        [ 38, 113, 448,  ..., 355, 651, 356]])
Validation Data Edge Index: tensor([[795, 639, 498,  ...,   6, 770, 297],
        [177, 549, 207,  ..., 355, 651, 356]])
Test Data Edge Index: tensor([[795, 639, 498,  ..., 915, 659, 663],
        [177, 549, 207,  ..., 133, 384, 195]])


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers=1, dropout=0.1):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.dropout = torch.nn.Dropout(dropout)

        # Input to hidden layer
        self.convs.append(SAGEConv((-1, -1), hidden_channels))

        # Hidden to hidden layers
        for _ in range(1, num_layers - 1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))

        # Hidden to output layer
        self.convs.append(SAGEConv(hidden_channels, hidden_channels))

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        for conv in self.convs[:-1]:
            x = F.relu(conv(x, edge_index))
            x = self.dropout(x)  # Apply dropout after each hidden layer

        x = self.convs[-1](x, edge_index)
        return x



class Model(torch.nn.Module):
    def __init__(self, data: HeteroData, hidden_channels):
        super().__init__()
        # Extract the number of nodes for users and movies
        num_users = data['user'].num_nodes
        num_movies = data['movie'].num_nodes
        user_feat_dim = data['user'].x.size(1)
        movie_feat_dim = data['movie'].x.size(1)

        # Learnable embeddings for users and movies
        self.user_emb = torch.nn.Embedding(num_users, hidden_channels)
        self.movie_emb = torch.nn.Embedding(num_movies, hidden_channels)

        # Project user and movie features to the hidden dimension
        self.user_lin = torch.nn.Linear(user_feat_dim, hidden_channels)
        self.movie_lin = torch.nn.Linear(movie_feat_dim, hidden_channels)

        self.gnn = GNN(hidden_channels)

        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

    def forward(self, data: HeteroData) -> dict:
        # Generate initial embeddings for users and movies
        x_dict = {
            "user": self.user_lin(data["user"].x) + self.user_emb.weight,  # Combine user features and embeddings
            "movie": self.movie_lin(data["movie"].x) + self.movie_emb.weight,  # Combine movie features and embeddings
        }

        # Apply the heterogeneous GNN
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        return x_dict


def bpr_loss(user_emb, pos_movie_emb, neg_movie_emb):
    pos_scores = (user_emb * pos_movie_emb).sum(dim=-1)
    neg_scores = (user_emb * neg_movie_emb).sum(dim=-1)
    loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()
    return loss


def train_bpr(data: HeteroData, model, optimizer):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    x_dict = model(data)
    user_emb = x_dict['user']
    movie_emb = x_dict['movie']

    # Positive and negative edge sampling
    pos_edge_index = data['user', 'rates', 'movie'].edge_index
    neg_edge_index = torch.randint(
        low=0, high=data['movie'].num_nodes, size=pos_edge_index.size(), device=pos_edge_index.device
    )

    # Extract embeddings for positive and negative edges
    user_emb = user_emb[pos_edge_index[0]]
    pos_movie_emb = movie_emb[pos_edge_index[1]]
    neg_movie_emb = movie_emb[neg_edge_index[1]]

    # Compute BPR loss
    loss = bpr_loss(user_emb, pos_movie_emb, neg_movie_emb)
    loss.backward()
    #print("User Embedding Gradients:", model.user_emb.weight.grad)
    #print("Movie Embedding Gradients:", model.movie_emb.weight.grad)
    optimizer.step()
    return loss.item()



def evaluate(data: HeteroData, model, k=20):
    model.eval()
    with torch.no_grad():
        x_dict = model(data)
        user_emb = x_dict['user']
        movie_emb = x_dict['movie']

        # Extract edge indices for positive edges
        pos_edge_index = data['user', 'rates', 'movie'].edge_index

        # Compute scores for all movies for each user
        scores = torch.matmul(user_emb, movie_emb.t())  # [num_users, num_movies]

        # Extract ground-truth positive edges
        user_ids = pos_edge_index[0]  # Users in positive edges
        movie_ids = pos_edge_index[1]  # Movies in positive edges

        # Initialize metrics
        precision = 0.0
        recall = 0.0
        ndcg = 0.0

        for user_id in user_ids.unique():
            user_scores = scores[user_id]  # Scores for the current user [num_movies]
            true_movie_ids = movie_ids[user_ids == user_id]  # Ground truth movies for the user

            # Skip users with no positive edges
            if true_movie_ids.numel() == 0:
                continue

            # Get top-K movie indices
            _, top_k_indices = torch.topk(user_scores, k)

            # Compute Precision@K
            hits = torch.isin(top_k_indices, true_movie_ids).sum().item()
            precision += hits / k

            # Compute Recall@K
            recall += hits / true_movie_ids.numel()

            # Compute nDCG@K
            gains = torch.isin(top_k_indices, true_movie_ids).float()
            discounts = torch.log2(torch.arange(2, k + 2).float())
            dcg = (gains / discounts).sum()
            idcg = (1.0 / discounts[: min(k, true_movie_ids.numel())]).sum()
            ndcg += dcg / idcg

        # Average metrics over all users
        num_users = user_ids.unique().numel()
        precision /= num_users
        recall /= num_users
        ndcg /= num_users

    return precision, recall, ndcg




# Initialize the model
hidden_channels = 64
model = Model(data, hidden_channels=hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 200
for epoch in range(num_epochs):
    train_loss = train_bpr(train_data, model, optimizer)
    precision, recall, ndcg = evaluate(val_data, model, k=20)

    print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Precision@20: {precision:.4f}, Recall@20: {recall:.4f}, nDCG@20: {ndcg:.4f}")


Epoch 0, Train Loss: 0.7524, Precision@20: 0.0249, Recall@20: 0.0096, nDCG@20: 0.0211
Epoch 1, Train Loss: 0.7122, Precision@20: 0.0284, Recall@20: 0.0105, nDCG@20: 0.0240
Epoch 2, Train Loss: 0.6782, Precision@20: 0.0324, Recall@20: 0.0126, nDCG@20: 0.0282
Epoch 3, Train Loss: 0.6539, Precision@20: 0.0356, Recall@20: 0.0139, nDCG@20: 0.0316
Epoch 4, Train Loss: 0.6290, Precision@20: 0.0415, Recall@20: 0.0163, nDCG@20: 0.0364
Epoch 5, Train Loss: 0.6091, Precision@20: 0.0466, Recall@20: 0.0182, nDCG@20: 0.0400
Epoch 6, Train Loss: 0.6006, Precision@20: 0.0499, Recall@20: 0.0200, nDCG@20: 0.0421
Epoch 7, Train Loss: 0.5763, Precision@20: 0.0534, Recall@20: 0.0219, nDCG@20: 0.0445
Epoch 8, Train Loss: 0.5698, Precision@20: 0.0578, Recall@20: 0.0238, nDCG@20: 0.0477
Epoch 9, Train Loss: 0.5539, Precision@20: 0.0641, Recall@20: 0.0264, nDCG@20: 0.0527
Epoch 10, Train Loss: 0.5518, Precision@20: 0.0688, Recall@20: 0.0284, nDCG@20: 0.0570
Epoch 11, Train Loss: 0.5294, Precision@20: 0.0748, R

k=3 hops

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers=3, dropout=0.1):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.dropout = torch.nn.Dropout(dropout)

        # Input to hidden layer
        self.convs.append(SAGEConv((-1, -1), hidden_channels))

        # Hidden to hidden layers
        for _ in range(1, num_layers - 1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))

        # Hidden to output layer
        self.convs.append(SAGEConv(hidden_channels, hidden_channels))

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        for conv in self.convs[:-1]:
            x = F.relu(conv(x, edge_index))
            x = self.dropout(x)  # Apply dropout after each hidden layer

        x = self.convs[-1](x, edge_index)
        return x



class Model(torch.nn.Module):
    def __init__(self, data: HeteroData, hidden_channels):
        super().__init__()
        # Extract the number of nodes for users and movies
        num_users = data['user'].num_nodes
        num_movies = data['movie'].num_nodes
        user_feat_dim = data['user'].x.size(1)
        movie_feat_dim = data['movie'].x.size(1)

        # Learnable embeddings for users and movies
        self.user_emb = torch.nn.Embedding(num_users, hidden_channels)
        self.movie_emb = torch.nn.Embedding(num_movies, hidden_channels)

        # Project user and movie features to the hidden dimension
        self.user_lin = torch.nn.Linear(user_feat_dim, hidden_channels)
        self.movie_lin = torch.nn.Linear(movie_feat_dim, hidden_channels)

        self.gnn = GNN(hidden_channels)

        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

    def forward(self, data: HeteroData) -> dict:
        # Generate initial embeddings for users and movies
        x_dict = {
            "user": self.user_lin(data["user"].x) + self.user_emb.weight,  # Combine user features and embeddings
            "movie": self.movie_lin(data["movie"].x) + self.movie_emb.weight,  # Combine movie features and embeddings
        }

        # Apply the heterogeneous GNN
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        return x_dict


def bpr_loss(user_emb, pos_movie_emb, neg_movie_emb):
    pos_scores = (user_emb * pos_movie_emb).sum(dim=-1)
    neg_scores = (user_emb * neg_movie_emb).sum(dim=-1)
    loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()
    return loss


def train_bpr(data: HeteroData, model, optimizer):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    x_dict = model(data)
    user_emb = x_dict['user']
    movie_emb = x_dict['movie']

    # Positive and negative edge sampling
    pos_edge_index = data['user', 'rates', 'movie'].edge_index
    neg_edge_index = torch.randint(
        low=0, high=data['movie'].num_nodes, size=pos_edge_index.size(), device=pos_edge_index.device
    )

    # Extract embeddings for positive and negative edges
    user_emb = user_emb[pos_edge_index[0]]
    pos_movie_emb = movie_emb[pos_edge_index[1]]
    neg_movie_emb = movie_emb[neg_edge_index[1]]

    # Compute BPR loss
    loss = bpr_loss(user_emb, pos_movie_emb, neg_movie_emb)
    loss.backward()
    #print("User Embedding Gradients:", model.user_emb.weight.grad)
    #print("Movie Embedding Gradients:", model.movie_emb.weight.grad)
    optimizer.step()
    return loss.item()



def evaluate(data: HeteroData, model, k=20):
    model.eval()
    with torch.no_grad():
        x_dict = model(data)
        user_emb = x_dict['user']
        movie_emb = x_dict['movie']

        # Extract edge indices for positive edges
        pos_edge_index = data['user', 'rates', 'movie'].edge_index

        # Compute scores for all movies for each user
        scores = torch.matmul(user_emb, movie_emb.t())  # [num_users, num_movies]

        # Extract ground-truth positive edges
        user_ids = pos_edge_index[0]  # Users in positive edges
        movie_ids = pos_edge_index[1]  # Movies in positive edges

        # Initialize metrics
        precision = 0.0
        recall = 0.0
        ndcg = 0.0

        for user_id in user_ids.unique():
            user_scores = scores[user_id]  # Scores for the current user [num_movies]
            true_movie_ids = movie_ids[user_ids == user_id]  # Ground truth movies for the user

            # Skip users with no positive edges
            if true_movie_ids.numel() == 0:
                continue

            # Get top-K movie indices
            _, top_k_indices = torch.topk(user_scores, k)

            # Compute Precision@K
            hits = torch.isin(top_k_indices, true_movie_ids).sum().item()
            precision += hits / k

            # Compute Recall@K
            recall += hits / true_movie_ids.numel()

            # Compute nDCG@K
            gains = torch.isin(top_k_indices, true_movie_ids).float()
            discounts = torch.log2(torch.arange(2, k + 2).float())
            dcg = (gains / discounts).sum()
            idcg = (1.0 / discounts[: min(k, true_movie_ids.numel())]).sum()
            ndcg += dcg / idcg

        # Average metrics over all users
        num_users = user_ids.unique().numel()
        precision /= num_users
        recall /= num_users
        ndcg /= num_users

    return precision, recall, ndcg




# Initialize the model
hidden_channels = 64
model = Model(data, hidden_channels=hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 200
for epoch in range(num_epochs):
    train_loss = train_bpr(train_data, model, optimizer)
    precision, recall, ndcg = evaluate(val_data, model, k=20)

    print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Precision@20: {precision:.4f}, Recall@20: {recall:.4f}, nDCG@20: {ndcg:.4f}")


Epoch 0, Train Loss: 0.6951, Precision@20: 0.0159, Recall@20: 0.0070, nDCG@20: 0.0132
Epoch 1, Train Loss: 0.6771, Precision@20: 0.0273, Recall@20: 0.0115, nDCG@20: 0.0227
Epoch 2, Train Loss: 0.6569, Precision@20: 0.0356, Recall@20: 0.0150, nDCG@20: 0.0291
Epoch 3, Train Loss: 0.6456, Precision@20: 0.0477, Recall@20: 0.0198, nDCG@20: 0.0390
Epoch 4, Train Loss: 0.6249, Precision@20: 0.0647, Recall@20: 0.0261, nDCG@20: 0.0544
Epoch 5, Train Loss: 0.6050, Precision@20: 0.0733, Recall@20: 0.0279, nDCG@20: 0.0644
Epoch 6, Train Loss: 0.5799, Precision@20: 0.0843, Recall@20: 0.0308, nDCG@20: 0.0753
Epoch 7, Train Loss: 0.5648, Precision@20: 0.0964, Recall@20: 0.0349, nDCG@20: 0.0857
Epoch 8, Train Loss: 0.5499, Precision@20: 0.1092, Recall@20: 0.0396, nDCG@20: 0.0963
Epoch 9, Train Loss: 0.5374, Precision@20: 0.1168, Recall@20: 0.0426, nDCG@20: 0.1052
Epoch 10, Train Loss: 0.5223, Precision@20: 0.1227, Recall@20: 0.0459, nDCG@20: 0.1167
Epoch 11, Train Loss: 0.5191, Precision@20: 0.1340, R

k=6

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers=6, dropout=0.1):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.dropout = torch.nn.Dropout(dropout)

        # Input to hidden layer
        self.convs.append(SAGEConv((-1, -1), hidden_channels))

        # Hidden to hidden layers
        for _ in range(1, num_layers - 1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))

        # Hidden to output layer
        self.convs.append(SAGEConv(hidden_channels, hidden_channels))

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        for conv in self.convs[:-1]:
            x = F.relu(conv(x, edge_index))
            x = self.dropout(x)  # Apply dropout after each hidden layer

        x = self.convs[-1](x, edge_index)
        return x



class Model(torch.nn.Module):
    def __init__(self, data: HeteroData, hidden_channels):
        super().__init__()
        # Extract the number of nodes for users and movies
        num_users = data['user'].num_nodes
        num_movies = data['movie'].num_nodes
        user_feat_dim = data['user'].x.size(1)
        movie_feat_dim = data['movie'].x.size(1)

        # Learnable embeddings for users and movies
        self.user_emb = torch.nn.Embedding(num_users, hidden_channels)
        self.movie_emb = torch.nn.Embedding(num_movies, hidden_channels)

        # Project user and movie features to the hidden dimension
        self.user_lin = torch.nn.Linear(user_feat_dim, hidden_channels)
        self.movie_lin = torch.nn.Linear(movie_feat_dim, hidden_channels)

        self.gnn = GNN(hidden_channels)

        self.gnn = to_hetero(self.gnn, metadata=data.metadata())

    def forward(self, data: HeteroData) -> dict:
        # Generate initial embeddings for users and movies
        x_dict = {
            "user": self.user_lin(data["user"].x) + self.user_emb.weight,  # Combine user features and embeddings
            "movie": self.movie_lin(data["movie"].x) + self.movie_emb.weight,  # Combine movie features and embeddings
        }

        # Apply the heterogeneous GNN
        x_dict = self.gnn(x_dict, data.edge_index_dict)
        return x_dict


def bpr_loss(user_emb, pos_movie_emb, neg_movie_emb):
    pos_scores = (user_emb * pos_movie_emb).sum(dim=-1)
    neg_scores = (user_emb * neg_movie_emb).sum(dim=-1)
    loss = -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()
    return loss


def train_bpr(data: HeteroData, model, optimizer):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    x_dict = model(data)
    user_emb = x_dict['user']
    movie_emb = x_dict['movie']

    # Positive and negative edge sampling
    pos_edge_index = data['user', 'rates', 'movie'].edge_index
    neg_edge_index = torch.randint(
        low=0, high=data['movie'].num_nodes, size=pos_edge_index.size(), device=pos_edge_index.device
    )

    # Extract embeddings for positive and negative edges
    user_emb = user_emb[pos_edge_index[0]]
    pos_movie_emb = movie_emb[pos_edge_index[1]]
    neg_movie_emb = movie_emb[neg_edge_index[1]]

    # Compute BPR loss
    loss = bpr_loss(user_emb, pos_movie_emb, neg_movie_emb)
    loss.backward()
    #print("User Embedding Gradients:", model.user_emb.weight.grad)
    #print("Movie Embedding Gradients:", model.movie_emb.weight.grad)
    optimizer.step()
    return loss.item()



def evaluate(data: HeteroData, model, k=20):
    model.eval()
    with torch.no_grad():
        x_dict = model(data)
        user_emb = x_dict['user']
        movie_emb = x_dict['movie']

        # Extract edge indices for positive edges
        pos_edge_index = data['user', 'rates', 'movie'].edge_index

        # Compute scores for all movies for each user
        scores = torch.matmul(user_emb, movie_emb.t())  # [num_users, num_movies]

        # Extract ground-truth positive edges
        user_ids = pos_edge_index[0]  # Users in positive edges
        movie_ids = pos_edge_index[1]  # Movies in positive edges

        # Initialize metrics
        precision = 0.0
        recall = 0.0
        ndcg = 0.0

        for user_id in user_ids.unique():
            user_scores = scores[user_id]  # Scores for the current user [num_movies]
            true_movie_ids = movie_ids[user_ids == user_id]  # Ground truth movies for the user

            # Skip users with no positive edges
            if true_movie_ids.numel() == 0:
                continue

            # Get top-K movie indices
            _, top_k_indices = torch.topk(user_scores, k)

            # Compute Precision@K
            hits = torch.isin(top_k_indices, true_movie_ids).sum().item()
            precision += hits / k

            # Compute Recall@K
            recall += hits / true_movie_ids.numel()

            # Compute nDCG@K
            gains = torch.isin(top_k_indices, true_movie_ids).float()
            discounts = torch.log2(torch.arange(2, k + 2).float())
            dcg = (gains / discounts).sum()
            idcg = (1.0 / discounts[: min(k, true_movie_ids.numel())]).sum()
            ndcg += dcg / idcg

        # Average metrics over all users
        num_users = user_ids.unique().numel()
        precision /= num_users
        recall /= num_users
        ndcg /= num_users

    return precision, recall, ndcg




# Initialize the model
hidden_channels = 64
model = Model(data, hidden_channels=hidden_channels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 200
for epoch in range(num_epochs):
    train_loss = train_bpr(train_data, model, optimizer)
    precision, recall, ndcg = evaluate(val_data, model, k=20)

    print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Precision@20: {precision:.4f}, Recall@20: {recall:.4f}, nDCG@20: {ndcg:.4f}")


Epoch 0, Train Loss: 0.6924, Precision@20: 0.0211, Recall@20: 0.0058, nDCG@20: 0.0172
Epoch 1, Train Loss: 0.6887, Precision@20: 0.0276, Recall@20: 0.0106, nDCG@20: 0.0213
Epoch 2, Train Loss: 0.6857, Precision@20: 0.0297, Recall@20: 0.0138, nDCG@20: 0.0230
Epoch 3, Train Loss: 0.6805, Precision@20: 0.0145, Recall@20: 0.0064, nDCG@20: 0.0121
Epoch 4, Train Loss: 0.6742, Precision@20: 0.0169, Recall@20: 0.0074, nDCG@20: 0.0133
Epoch 5, Train Loss: 0.6624, Precision@20: 0.0151, Recall@20: 0.0062, nDCG@20: 0.0116
Epoch 6, Train Loss: 0.6504, Precision@20: 0.0112, Recall@20: 0.0050, nDCG@20: 0.0088
Epoch 7, Train Loss: 0.6375, Precision@20: 0.0104, Recall@20: 0.0034, nDCG@20: 0.0082
Epoch 8, Train Loss: 0.6121, Precision@20: 0.0094, Recall@20: 0.0031, nDCG@20: 0.0075
Epoch 9, Train Loss: 0.6224, Precision@20: 0.0266, Recall@20: 0.0086, nDCG@20: 0.0198
Epoch 10, Train Loss: 0.5999, Precision@20: 0.0350, Recall@20: 0.0126, nDCG@20: 0.0275
Epoch 11, Train Loss: 0.6047, Precision@20: 0.0539, R